In [ ]:
import config
from phemex import *
import talib
import json
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import nest_asyncio

nest_asyncio.apply()
import asyncio
import websockets
clients = {}
# Create an instance of the Phemex exchange
exchange = initExchange()
symbol = config.TRADE_SYMBOL
trades = pd.DataFrame(columns=['timestamp', 'side', 'priceEp', 'volume'])
tradesPrices = []
ohlcvData = pd.DataFrame(
    columns=['open', 'high', 'low', 'close', 'volume', 'timestamp'])
klineData = pd.DataFrame(
    columns=['timestamp', 'interval', 'lastCloseEp', 'openEp', 'highEp', 'lowEp', 'closeEp', 'volume', 'turnoverEv'])


In [ ]:
def create_or_update_stoploss(symbol, side, entry_price, stop_loss_price, amount):
    # Check if there is already an open stoploss order for this symbol and side
    existing_orders = exchange.fetch_open_orders(symbol=symbol)
    stoploss_order = None
    order_side = None
    difference = abs(entry_price - stop_loss_price)
    if side == 'Buy':
        order_side = 'Sell'
        if difference < 10:
            stop_loss_price - 10
    if side == 'Sell':
        order_side = 'Buy'
        if difference < 10:
            stop_loss_price + 10
    # Filter out only stoploss orders
    for order in existing_orders:
        if order['info']['orderType'] == 'Stop' and order['info']['side'] == order_side:
            stoploss_order = order
            break

    if stoploss_order is None:
        # Create a new stoploss order
        print(f"Create a new stoploss order")
        order = exchange.create_order(
            symbol=symbol,
            type='stop',
            side=order_side,
            amount=amount,
            params={
                'stopPx': stop_loss_price,
                'ordType': 'Stop',
                'execInst':'LastPrice',
                'px': entry_price,
            },
        )
        return order
    else:
        # Update the existing stoploss order
        print(f"Update the existing stoploss order")
        order = exchange.edit_order(
            id=stoploss_order['id'],
            symbol=symbol,
            amount=amount,
            params={
                'stopPx': stop_loss_price,
                'ordType': 'Stop',
                'execInst': 'LastPrice',
                'px': entry_price,
            },
        )
        return order

In [ ]:
import ccxt
import time
import ta

def stoplossStrategy(ohlcvData,current_price):
    stop_loss_price = 0
    position = getPositionInfo()[0]
    position_size = position['amount']
    leverage = position['leverage']
    unrealized_pnl = position['unrealized_pnl']
    side = position['side']
    # Set the trading parameters
    symbol = 'BTCUSD'
    stop_loss_percent = 0.05 / leverage
    limit_order_percent = 0.01 / leverage
    quantity = 4
    # Get the current market price
    # current_price = ticker['priceEp']

    # Calculate the Parabolic SAR
    psar_indicator = ta.trend.PSARIndicator(high=ohlcvData['high'], low=ohlcvData['low'], close=ohlcvData['close'])
    ohlcvData['sar'] = psar_indicator.psar()
    
    # Define a function to determine the trend on Parabolic SAR
    def get_sar_trend(sar_values):
        trend = []
        for i in range(len(sar_values)):
            if i == 0:
                trend.append('N/A')
            elif sar_values[i] > sar_values[i-1]:
                trend.append('bullish')
            else:
                trend.append('bearish')
        return trend

    # Determine the trend on Parabolic SAR
    ohlcvData['sar_trend'] = get_sar_trend(ohlcvData['sar'])
    # Calculate the percentage change in the SAR value
    ohlcvData['sar_change'] = ohlcvData['sar'].pct_change() * leverage   
    currentSar = ohlcvData.iloc[-1]['sar']
    currentSma200 = ohlcvData.iloc[-1]['sma200']
    # Calculate the buy and sell signals
    ohlcvData['buy_signal'] = (ohlcvData['close'] > ohlcvData['sar']) & (ohlcvData['close'] > ohlcvData["sma200"])  #(ohlcvData['sar_change'] > 0.02)
    ohlcvData['sell_signal'] = (ohlcvData['close'] < ohlcvData['sar']) & (ohlcvData['close'] < ohlcvData["sma200"]) #(ohlcvData['sar_change'] < -0.02)
    currentBuySignal = (current_price > currentSar) & (current_price > currentSma200)  #(ohlcvData['sar_change'] > 0.02)
    currentSellSignal = (current_price < currentSar) & (current_price < currentSma200) #(ohlcvData['sar_change'] < -0.02)    
    # print(f'current_price {round(current_price * 0.0001,0)} currentSar {round(currentSar * 0.0001,0)} currentSma200 {round(currentSma200 * 0.0001,0)} currentBuySignal {currentBuySignal} currentSellSignal {currentSellSignal}')
    # time.sleep(800060)
    # display(ohlcvData.iloc[-1])
    existing_orders = exchange.fetch_open_orders(symbol=symbol)  
    limitFound = False 
    stoplossFound = False     
    for order in existing_orders:
        print('Existing orders: type={} side={}'.format(order['info']['orderType'],order['info']['side']))
        if order['info']['orderType'] == 'Limit' and not limitFound:
            side = order['info']['side']
            # print(f'Stopploss order: {stoploss_order}')
            limitFound = True  
        if order['info']['orderType'] == 'Stop' and not stoplossFound:
            side = order['info']['side']
            stoploss_order = order
            # print(f'Stopploss order: {stoploss_order}')
            stoplossFound = True
    print(f'ohlcv data current price {round(current_price * 0.0001,0)} sma200 {round(ohlcvData.iloc[-1]["sma200"] * 0.0001,0)} fib[0.236] {round(ohlcvData.iloc[-1]["fib_0.236"] * 0.0001,0)}')
    if not limitFound and side == 'None':
        if currentSellSignal:
            print('Sar signal is Sell',ohlcvData.iloc[-1])
            # Set the stop-loss and limit order prices
            stop_loss_price = ohlcvData.iloc[-1]["fib_0.236"] #current_price * (1 + stop_loss_percent)
            limit_order_price = current_price - 5 #* (1 - limit_order_percent)
            print(f'current_price: {round(current_price * 0.0001,0)} stop_loss_price: {stop_loss_price * 0.0001} limit_order_price: {limit_order_price * 0.0001}')

            # Place the market order
            limit_order = exchange.create_order(symbol=symbol, type='limit', side='sell', amount=quantity, price=limit_order_price * 0.0001)
            print(f'created sell limit_order: {limit_order}')
            # time.sleep(800060)
                     
        if currentBuySignal:
            print('Sar signal is Buy',ohlcvData.iloc[-1])
            # Set the stop-loss and limit order prices
            stop_loss_price = ohlcvData.iloc[-1]["fib_0.236"]
            limit_order_price = current_price + 5 #* (1 + limit_order_percent)
            print(f'current_price: {round(current_price * 0.0001,0)} stop_loss_price: {stop_loss_price * 0.0001} limit_order_price: {limit_order_price * 0.0001}')

            # Place the market buy orders
            limit_order = exchange.create_order(symbol=symbol, type='limit', side='buy', amount=quantity, price=limit_order_price * 0.0001)
            print(f'created buy limit_order: {limit_order}')
            # time.sleep(800060)
    
            

    # Print the last few rows of the data
    # display(ohlcvData.tail())
    # Monitor the position and adjust the stop-loss and limit orders as necessary
    # while True:
    position = getPositionInfo()[0]
    position_size = position['amount']
    side = position['side']
    unrealized_pnl = position['unrealized_pnl']
    unrealized_pnl_perc = position['unrealized_pnl_perc']
    entryPrice = position['entryPrice']
    if stoplossFound and side != 'None':
        print(f'side: {side}')
        # Check if there is already an open stoploss order for this symbol and side
        if side == 'Buy':
            # If the unrealized_pnl_perc is more than 20% cancel position
            if unrealized_pnl_perc > 20:
                try:
                    exchange.cancel_order(stoploss_order['id'], symbol)                  
                    print('Remove Stoploss')
                    cancelResponse = exchange.cancel_all_orders(symbol,params={'untriggered':True})
                    print('All stoploss orders removed',cancelResponse)
                    stoploss_order = None
                except Exception as e:
                    print('Error {}'.format(e))
                    pass  
        if side == 'Sell':
            # If the unrealized_pnl_perc is less than 20% cancel position
            if unrealized_pnl_perc < -20:
                try:
                    exchange.cancel_order(stoploss_order['id'], symbol)                  
                    print('Remove Stoploss')
                    cancelResponse = exchange.cancel_all_orders(symbol,params={'untriggered':True})
                    print('All stoploss orders removed',cancelResponse)
                    stoploss_order = None
                except Exception as e:
                    print('Error {}'.format(e))
                    pass                  
        if stoploss_order != None:        
            # difference = abs(entryPrice - stop_loss_price)
            stop_loss_price = ohlcvData.iloc[-1]["fib_0.236"] * 0.0001 #current_price * (1 - stop_loss_percent)
            
            # Update the stop-loss and limit orders
            print(f'stop order {stoploss_order}')
            exchange.edit_order(id=stoploss_order['id'], symbol=symbol, price=stop_loss_price)
            
            # Check if the position is in danger of being liquidated
            # print(f'Position in danger: unrealized_pnl: {unrealized_pnl} < position_size: {position_size} * stop_loss_percent: {stop_loss_percent}%')
            # print(f'Position in danger: {unrealized_pnl} <  {position_size * stop_loss_percent}')
            # if unrealized_pnl < position_size * stop_loss_percent:
            #     # If the unrealized PnL is less than the stop-loss threshold, trigger the stop-loss order
            #     exchange.cancel_order(stoploss_order['id'], symbol)
            #     exchange.create_order(symbol, 'market', 'sell', quantity)
                    
                # # Sleep for a period of time before repeating the loop
                # time.sleep(60)
    else:
        if side != 'None':
            # Set the stop-loss order prices
            stop_loss_price = ohlcvData.iloc[-1]["fib_0.236"]    
            if side == 'Buy':        
                params = {
                        "orderType": "Stop",
                        "timeInForce": "ImmediateOrCancel",
                        "trigger": "ByMarkPrice",
                        "stopDirection": "Rising",
                        "stopPxEp": stop_loss_price,
                        "stopPx": stop_loss_price * 0.0001,
                        "execInst": "CloseOnTrigger",
                }
                stoploss_order = exchange.create_order(symbol=symbol, type='stop', side='sell', amount=quantity, params=params)
            if side == 'Sell':
                # Place the stop-loss and limit orders
                params = {
                        "orderType": "Stop",
                        "timeInForce": "ImmediateOrCancel",
                        "trigger": "ByMarkPrice",
                        "stopDirection": "Falling",
                        "stopPxEp": stop_loss_price,
                        "stopPx": stop_loss_price * 0.0001,
                        "execInst": "CloseOnTrigger",
                }            
                stoploss_order = exchange.create_order(symbol=symbol, type='stop', side='buy', amount=quantity, params=params)
            


In [ ]:
def getTimeformat(interval):
    if 'S' in interval:
        timeformat = "%Y-%m-%d, %H:%M:%S"
    elif 'min' in interval:
        timeformat = "%Y-%m-%d, %H:%M"
    elif 'H' in interval:
        timeformat = "%Y-%m-%d, %H"
    elif 'D' in interval:
        timeformat = "%Y-%m-%d"
    elif 'M' in interval:
        timeformat = "%Y-%m"
    elif 'Y' in interval:
        timeformat = "%Y"
    else:
        timeformat = "%Y-%m-%d, %H:%M:%S"
    return timeformat

In [ ]:
async def getTradePrices(message, interval):
    # print('-- Function getTradePrices --')
    global trades
    try:
        if len(trades) > 0:
            trades.append(json.loads(message)['trades'][0])
        else:
            trades = list(json.loads(message)['trades'])
            
        df = pd.DataFrame(trades,columns=['timestamp', 'side', 'priceEp', 'volume'])
        tick_datetime_object = pd.to_datetime(df['timestamp'], unit='ns', utc=True)
        timenow = pd.to_datetime(tick_datetime_object + timedelta(hours=1))
        df['timestamp'] = timenow
        df.set_index('timestamp', inplace=True)
        df.reset_index()
        tradesAgg = df.groupby([pd.Grouper(level='timestamp',
                                           freq=interval)]).agg({
                                               'side':
                                               lambda x: list(x),
                                               'priceEp':
                                               'max',
                                               'volume':
                                               'sum'
                                           })

        return tradesAgg.dropna()
    except Exception as e:
        print('Foutmelding in functie getTradePrices: {}'.format(e))


In [ ]:
def addFibbLevels(ohlcv):
    # Calculate Fibonacci retracement levels
    ohlcv = ohlcv.copy().tail(380)
    high = ohlcv['high'].max()
    low = ohlcv['low'].min()
    # low = ohlcv.iloc[-1]['sma200']
    diff = high - low
    ohlcv['fib_0.0'] = low
    ohlcv['fib_0.236'] = low + 0.236 * diff
    ohlcv['fib_0.5'] = low + 0.5 * diff
    ohlcv['fib_0.618'] = low + 0.618 * diff
    ohlcv['fib_0.65'] = low + 0.65 * diff
    ohlcv['fib_0.786'] = low + 0.786 * diff
    ohlcv['fib_1.0'] = high
    ohlcv['fib_1.618'] = high + 1.618 * diff
    print('fib_0.0 {} fib_0.236 {} fib_0.5 {} fib_0.618 {} fib_0.65 {} fib_0.786 {} fib_1.0 {} fib_1.618 {}'.format(ohlcv.iloc[-1]['fib_0.0'] * 0.0001,ohlcv.iloc[-1]['fib_0.236'] * 0.0001,ohlcv.iloc[-1]['fib_0.5'] * 0.0001,ohlcv.iloc[-1]['fib_0.618'] * 0.0001,ohlcv.iloc[-1]['fib_0.65'] * 0.0001,ohlcv.iloc[-1]['fib_0.786'] * 0.0001,ohlcv.iloc[-1]['fib_1.0'] * 0.0001,ohlcv.iloc[-1]['fib_1.618'] * 0.0001))
    # Add current price and signal
    current_price = ohlcv.iloc[-1]['close']
    ohlcv['signal'] = 'none'

    # Generate buy/sell signal
    if (current_price < ohlcv['fib_0.236'].iloc[-1]):
        ohlcv['signal'] = 'Buy'
    elif (current_price > ohlcv['fib_0.618'].iloc[-1]
          and current_price < ohlcv['fib_0.65'].iloc[-1]):
        ohlcv['signal'] = 'Sell'

    return ohlcv

In [ ]:
async def getOhlc(klines, interval='1min'):
    global ohlcvData
    try:
        # print('-- Function getOhlc --')
        ohlc = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume', 'sma200', 'timestamp'])
        ohlcTemp = klines['closeEp'].resample(interval).ohlc()
        ohlcTemp.dropna() 
        ohlc['close'] = (ohlcTemp['open'] + ohlcTemp['high'] + ohlcTemp['low'] + ohlcTemp['close']) / 4
        ohlc['open'] = (ohlcTemp['open'].shift(1) + ohlcTemp['close'].shift(1)) / 2
        ohlc['high'] = ohlcTemp[['high', 'open', 'close']].max(axis=1)
        ohlc['low'] = ohlcTemp[['low', 'open', 'close']].min(axis=1)        
        ohlc['timestamp'] = ohlc.index.astype(np.int64) // 10**9
        ohlc['volume'] = klines['volume'].resample(interval).mean()
        ohlc['sma200'] = ohlc['close'].rolling(window=20).mean()
        ohlc = ohlc[ohlc['sma200'] > 0]
        ohlcvData = pd.concat([ohlcvData, ohlc])
        ohlcvData = addFibbLevels(ohlcvData)
        ohlcvData.dropna()        
        return ohlcvData
    except Exception as e:
        print('Foutmelding in functie getOhlc: {}'.format(e))

In [ ]:
async def getLastPrices(message):
    if len(message) > 0:
        trades.append(json.loads(message)['kline'][0])
    else:
        trades = list(json.loads(message)['kline'])

In [ ]:
async def send_message(message):
    
        global ohlcvData, tradesPrices, klineData
        if message != None:
            if 'kline' in message:
                temp = pd.DataFrame(json.loads(message)['kline'],columns=['timestamp', 'interval', 'lastCloseEp', 'openEp', 'highEp', 'lowEp', 'closeEp', 'volume', 'turnoverEv'])
                if json.loads(message)['type'] == 'snapshot':
                    klineData = temp
                else:
                    klineData = pd.concat([klineData,temp])
                klineData.dropna()
                klineData['datetime'] = pd.to_datetime(klineData['timestamp'],unit='s')
                klineData = klineData.set_index('datetime')
                # print(f'klineData length: {len(klineData)} lastcloseEp: {klineData.iloc[-1]["lastCloseEp"]} closeEp: {klineData.iloc[-1]["closeEp"]}')
                if len(klineData) > 900:
                    ohlcvData = await getOhlc(klineData, '1min')
                    # display(ohlcvData)
                    currentPrice = klineData.sort_index(ascending=False).iloc[0]["closeEp"]
                    try:
                        stoplossStrategy(ohlcvData,currentPrice)
                    except Exception as e:
                        print('Error in stoplossStrategy: {}'.format(e))  
                        pass                      
                    for clientId in clients:
                        client = clients[clientId]
                        print(ohlcvData.to_json(orient='records'))
                        sendMessage = json.dumps({'ohlcv':ohlcvData.reset_index().to_json(orient='records')})
                        await client['websocket'].send(sendMessage)


In [ ]:
# connect to Phemes websocket server
async def connectPhemexWS():
    global interval
    async for websocket in websockets.connect('wss://vapi.phemex.com/ws'):
        try:
            print('Phemex Websocket Server is connected!')
            # subscribe_ticker_msg = json.dumps({
            #     "id": 0,
            #     "method": "tick.subscribe",
            #     "params": [config.MBTC_SYMBOL]
            # })
            # await websocket.send(subscribe_ticker_msg)

            # subscribe_trade_msg = json.dumps({
            #     "id": 0,
            #     "method": "trade.subscribe",
            #     "params": [config.TRADE_SYMBOL]
            # })
            # await websocket.send(subscribe_trade_msg)

            subscribe_kline_msg = json.dumps({
                "id": 0,
                "method": "kline.subscribe",
                "params": [config.TRADE_SYMBOL_BTCUSD,60]
            })
            await websocket.send(subscribe_kline_msg)

            # subscribe_orderbook_msg = json.dumps({
            #     "id":
            #     0,
            #     "method":
            #     "orderbook.subscribe",
            #     "params": [config.TRADE_SYMBOL, True]
            # })
            # await websocket.send(subscribe_orderbook_msg)

            while True:
                message = await websocket.recv()
                # if 'book' in message:
                #     print(message)
                await send_message(message)
        except websockets.ConnectionClosed:
            continue

In [ ]:
async def register(websocket):
    # Assign a unique ID to the client
    client_id = len(clients) + 1
    clients[client_id] = {"websocket": websocket, "info": {}}
    # await websocket.send(f"You are client {client_id}")
    print(f"Client {client_id} connected")
    return client_id

async def unregister(client_id):
    # Remove the client from the clients array
    del clients[client_id]
    print(f"Client {client_id} disconnected")

async def handle_message(client_id, message):
    # Handle incoming messages from the client
    print(f"Received message from client {client_id}: {message}")
    # Store extra client information in clients array
    clients[client_id]["info"] = {"extra_info": "example"}

async def handle_client(websocket, path):
    client_id = await register(websocket)
    try:
        async for message in websocket:
            await handle_message(client_id, message)
    except websockets.exceptions.ConnectionClosed:
        pass
    finally:
        # Unregister the client when they disconnect
        await unregister(client_id)

In [ ]:
# start python localhost websocket server
async def start_server():
    try:
        async with websockets.serve(handle_client, host="localhost", port=8765):
            print("Server started")
            await connectPhemexWS()
            await asyncio.Future()  # Keep the server running
    except Exception as e:
        print('Error {}'.format(e))

In [ ]:
if __name__ == '__main__':
    asyncio.run(start_server())